In [1]:
import os
import csv
from supabase import create_client, Client
import os
from dotenv import load_dotenv
from pathlib import Path

env_path = Path('..') / '.env.local'
load_dotenv(dotenv_path=env_path)

url = os.getenv("NEXT_PUBLIC_SUPABASE_URL")
key = os.getenv("NEXT_PUBLIC_SUPABASE_ANON_KEY")
supabase: Client = create_client(url, key)

data = supabase.table("league").select("*").execute()
print(data)

data=[{'league_unique': 'ncaambb', 'league_name': "NCAA Men's Basketball", 'sport': 'basketball', 'womens': False}] count=None


In [13]:
def update_game_schedule(game_schedule_csv):
    game_insert = []
    with open(game_schedule_csv, 'r', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        next(reader)
        for row in reader:
            game_insert.append(row)

    data = supabase.table("game").upsert(game_insert,
                                         # ignore_duplicates=True, on_conflict="game_date, team_1_id, team_2_id"
                                         ).execute()
    print(data)

In [14]:
update_game_schedule("game-schedule-2023-ncaa-tournament.csv")

data=[{'game_id': 1, 'game_date': '2023-03-16', 'team_1_id': 'virginia', 'team_2_id': 'furman', 'game_time': '12:40:00', 'round_num': 2, 'competition_id': 1}, {'game_id': 2, 'game_date': '2023-03-16', 'team_1_id': 'missouri', 'team_2_id': 'utah-state', 'game_time': '13:40:00', 'round_num': 2, 'competition_id': 1}, {'game_id': 3, 'game_date': '2023-03-16', 'team_1_id': 'kansas', 'team_2_id': 'howard', 'game_time': '14:00:00', 'round_num': 2, 'competition_id': 1}, {'game_id': 4, 'game_date': '2023-03-16', 'team_1_id': 'alabama', 'team_2_id': 'texas-am-corpus-christi', 'game_time': '14:45:00', 'round_num': 2, 'competition_id': 1}, {'game_id': 5, 'game_date': '2023-03-16', 'team_1_id': 'san-diego-state', 'team_2_id': 'college-of-charleston', 'game_time': '15:10:00', 'round_num': 2, 'competition_id': 1}, {'game_id': 6, 'game_date': '2023-03-16', 'team_1_id': 'arizona', 'team_2_id': 'princeton', 'game_time': '16:10:00', 'round_num': 2, 'competition_id': 1}, {'game_id': 7, 'game_date': '2023-